In [2]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np


# In[2]:


########################################
########### IMPORT DATA ################
########################################

#list of teams that are included in the analysis
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}


W=set(list(range(0,13))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
#set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)


# In[3]:


week = 13


# In[4]:


########################################
########### MODEL ######################
########################################

# Create an empty model
m= gp.Model(name="NFL2b")
# ADD DECISION VARIABLES HERE
x_var = m.addVars(T, T, W, vtype = GRB.BINARY, name = "X")


# In[10]:


### #### CONSTRAINTS & OBJ FUNCTIONS #######
########################################

# ADD CONSTRAINTS
# add constraints that The season was limited to 12 weeks
m.addConstrs(sum(sum(x_var[i,j,k]+x_var[j,i,k] for k in W) for i in T) == 12 for j in T)
# add constraints that Each team would play once per week
#m.addConstrs(sum(x_var[i,j,k]+x_var[j,i,k] for j in T) == 1 for i in T for k in range(0,8))
#m.addConstrs(sum(x_var[i,j,k]+x_var[j,i,k] for j in T) == 1 for i in T for k in range(9,13))
# add constraints that All 12 games that a team played would need to be against a different opponent
m.addConstrs(sum(x_var[i,j,k]+x_var[j,i,k]for k in W) <= 1 for i in T for j in T)
# add constraints that Each team would play at most six home games (i.e., on their home stadium)
m.addConstrs(sum(sum(x_var[i,j,k] for k in W) for j in T) <=6 for i in T)
# add constraints to avoid circumstances that i=i
m.addConstrs(x_var[i,i,k] == 0 for i in T for i in T for k in W)
# add constraints to let binary variables are bound between 0 and 1
m.addConstrs(x_var[i,j,k] <= 1 for i in T for j in T for k in W)
m.addConstrs(x_var[i,j,k] >= 0 for i in T for j in T for k in W)

# add gaming pattern constraints for 2a
m.addConstrs(sum(x_var[i,j,k]+x_var[i,j,k+1]+x_var[i,j,k+2] for j in T) <= 2 for i in T for k in range(week-2))
m.addConstrs(sum(x_var[i,j,k]+x_var[i,j,k+1]+x_var[i,j,k+2] for j in T) >= 1 for i in T for k in range(week-2))

# add gaming pattern constraints for there are no games on week9
m.addConstrs(x_var[i,j,8] == 0 for i in T for j in T)

# ADD OBJECTIVE FUNCTION
# Objective is to minimize the total travel distance of all teams
obj = sum(E.iloc[i,j]*x_var[i,j,k] for i in T for j in T for k in W)*2

#optimize the model
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()


# In[6]:


########################################
########### PRINT RESULTS ##############
########################################

# ADD PRINTING
# Print optimal value of the objective function
print('\nValue of objective function: %g' % m.objVal)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 24192 rows, 7488 columns and 97872 nonzeros
Model fingerprint: 0xa220f5b1
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 23436 rows and 1416 columns
Presolve time: 0.26s
Presolved: 756 rows, 6072 columns, 50232 nonzeros
Variable types: 0 continuous, 6072 integer (6072 binary)

Root relaxation: objective 1.659080e+05, 1696 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    165908.00000 165908.000  0.00%     -    0s

Explored 0 nodes (2910 simplex iterations) in 0.63 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 165908 

Optimal solution found (t